In [199]:
import itertools 
import random
#subset_sum takes a list of numbers and calculates all possible combination of those list of numbers that would add up to the target number
#this is used to calculate which cards would be played that would maximize mana usage
def subset_sum(numbers, target, partial=[], partial_sum=0):
    if partial_sum == target:
        yield partial
    if partial_sum >= target:
        return
    for i, n in enumerate(numbers):
        remaining = numbers[i + 1:]
        yield from subset_sum(remaining, target, partial + [n], partial_sum + n)
#power_spike_turn is the number of turns you want to simulate to find the curve with the greatest amount of pressure against the opponent
#games_per_curve is the amount of games you simulate per curve
def optimalcurve(power_spike_turn, games_per_curve):
    #curvelist generates all possible curves for a deck given the turn you are simulating up to
    #so if you are simulating up to only turn 6, the deck would not contain cards higher that 6 mana
    curvelist=[i for i in itertools.combinations_with_replacement([u for u in range(power_spike_turn+1)], 30)] 
    #curvepressurelist holds the average pressure generated from each simulated curve
    curvepressurelist=[]
    #we go through all possible curves
    for j in range(len(curvelist)):
        #pressurelist holds the pressures generated for each game simulated
        pressurelist=[]
        #we go through each game
        for k in range(games_per_curve):
            #decklist obtains the list of cards
            decklist=list(curvelist[j])
            #we shuffle the deck
            random.shuffle(decklist)
            #we draw the first 3 cards from our deck to our hand
            hand=decklist[:3]
            #we remove the first 3 cards from our deck
            decklist=decklist[3:]
            #we create a list to hold the cards we want to mulligan
            mulligan=[]
            #we go through our hand
            for m in range(len(hand)):
                #if there is a card in our deck that is lower cost that a card in our hand and the card in our hand is not 0,1,2 cost we mulligan
                if min(decklist)<hand[m] and hand[m]!=0 and hand[m]!=1 and hand[m]!=2:
                    mulligan.append(hand[m])
            for r in range(len(mulligan)):
                #we add the mulligan cards to our deck
                decklist.append(mulligan[r]) 
                #we remove the mulligan cards from our hand
                hand.remove(mulligan[r])
            #we shuffle the deck
            random.shuffle(decklist)
            #we draw the amount of cards we mulligan from our hand
            for d in range(len(mulligan)):
                hand.append(decklist[0])
                decklist.pop(0)
            #we shuffle the deck
            random.shuffle(decklist)
            #manacount holds the total mana spent during a game
            manacount=0
            #pressure is calculated by adding the total amount of mana spent during the entire game each turn
            #so if you play a 1 mana card on turn one, the total pressure of that card would be 1 on turn 2, 2 on turn 2, 3 on turn 3, etc
            #this is to account for tempo
            #We assume all cards are worth their mana cost and your opponent can equalize the pressure generated by playing a 1 mana card also
            pressure=0
            #we go through each turn
            for t in range(power_spike_turn):
                #we draw a card from our deck
                hand.append(decklist[0])
                decklist.pop(0)
                #if a 0 cost card is in our hand we play it and we say that the 0 cost card is worth 0.5 of a 1 cost card
                #This means it will generate 0.5 points of pressure each turn thus we add 0.5 points into mana spent
                while 0 in hand:
                    hand.remove(0)
                    manacount+=0.5
                #we go through all possible card combinations we can play on that turn that would maximize mana usage
                for p in range(t+1):
                    #we sort the list so we can pick the combination that would allow us to prioritize playing the highest cost card
                    #example would be playing a 6 cost card on turn 6 instead of two 3 cost cards
                    cardstoplay=sorted(list(subset_sum(hand,t+1-p)), reverse=True)
                    #if there exists card combinations we can play on this turn
                    if cardstoplay!=[]:
                        #we play the cards and add them to total mana spent
                        for c in range(len(cardstoplay[0])):
                            hand.remove(cardstoplay[0][c])
                            manacount+=cardstoplay[0][c]
                        break
                #we add total mana spent each turn to pressure
                pressure+=manacount
            #we add the pressure generated in each game to the list of pressures for the curve
            pressurelist.append(pressure)
        #we take the average pressure generated for that curve and at it to the list holding the average pressure for all curves
        curvepressurelist.append(sum(pressurelist)/len(pressurelist))
    #we find the curve that generated the highest amount of pressure and print it
    print(curvelist[curvepressurelist.index(max(curvepressurelist))])
    #we find the average pressure generated for that curve and print it
    print('average pressure = '+str(curvepressurelist[curvepressurelist.index(max(curvepressurelist))]))